In [1]:
# 🔹 2. นำเข้าไลบรารี
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments

c:\Users\MSI T\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ตรวจสอบ CPU, GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [3]:
# 🔹 3. โหลดชุดข้อมูล (Dataset)
dataset = load_dataset("Thaweewat/alpaca-cleaned-52k-th")

In [4]:
# ดูตัวอย่างข้อมูล
print(dataset["train"][0])  # ตัวอย่างข้อมูลจากชุดข้อมูล train

{'instruction': ' ให้สามเคล็ดลับในการรักษาสุขภาพ', 'input': '', 'output': ' 1. รับประทานอาหารที่สมดุลและมีคุณค่าทางโภชนาการ: ตรวจสอบให้แน่ใจว่ามื้ออาหารของคุณประกอบด้วยผักและผลไม้หลากหลายชนิด โปรตีนไม่ติดมัน เมล็ดธัญพืช และไขมันที่ดีต่อสุขภาพ สิ่งนี้ช่วยให้ร่างกายของคุณได้รับสารอาหารที่จำเป็นเพื่อให้ทำงานได้ดีที่สุดและสามารถช่วยป้องกันโรคเรื้อรังได้ 2. ออกกำลังกายอย่างสม่ำเสมอ: การออกกำลังกายเป็นสิ่งสำคัญสำหรับการรักษากระดูก กล้ามเนื้อ และสุขภาพหัวใจและหลอดเลือดให้แข็งแรง ตั้งเป้าหมายให้ออกกำลังกายแบบแอโรบิกระดับปานกลางอย่างน้อย 150 นาทีหรือออกกำลังกายแบบหนัก 75 นาทีต่อสัปดาห์ 3. นอนหลับให้เพียงพอ: การนอนหลับที่มีคุณภาพเพียงพอเป็นสิ่งสำคัญสำหรับความเป็นอยู่ที่ดีทั้งทางร่างกายและจิตใจ ช่วยควบคุมอารมณ์ ปรับปรุงการทำงานของสมอง และสนับสนุนการเจริญเติบโตอย่างแข็งแรงและการทำงานของภูมิคุ้มกัน ตั้งเป้าหมายการนอนหลับให้ได้ 7-9 ชั่วโมงในแต่ละคืน'}


In [5]:
# แบ่งข้อมูล train เป็น train และ validation (80% train, 20% validation)
train_dataset = dataset["train"].train_test_split(test_size=0.2)
val_dataset = train_dataset["test"]
train_dataset = train_dataset["train"]

In [6]:
print(train_dataset)
print(val_dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 41408
})
Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 10352
})


In [7]:
# 🔹 4. โหลด Base Model และ Tokenizer (ใช้ GPU)
model_name = "Pavarissy/phayathaibert-thainer"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

c:\Users\MSI T\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MSI T\.cache\huggingface\hub\models--Pavarissy--phayathaibert-thainer. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
If you want to use `CamembertLMHeadModel` as a standalone, add `is_decoder=True.`


In [8]:
# ทดสอบ model
input_text = "กินข้าวดีไหม"

# ทำการ tokenize ข้อความ input
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(device)

# ใช้โมเดลในการสร้าง output
with torch.no_grad():  # ไม่ต้องคำนวณ gradient
    output = model.generate(inputs["input_ids"], max_length=50, num_beams=5, early_stopping=True)

# แปลง output ที่เป็น token กลับเป็นข้อความ
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

# แสดงผลลัพธ์
print(f"Input: {input_text}")
print(f"Output: {output_text}")

c:\Users\MSI T\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:2137: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
# 🔹 5. Tokenization ข้อมูล
def tokenize_function(examples):
    # รวมข้อมูล instruction และ input เข้าด้วยกันในแต่ละตัวอย่าง
    inputs = [ex[0] + " " + ex[1] for ex in zip(examples["instruction"], examples["input"])]
    
    # ทำ tokenization สำหรับ inputs
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)

    # ทำ tokenization สำหรับ output (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["output"], padding="max_length", truncation=True, max_length=128)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# ใช้ map เพื่อแปลงข้อมูลทั้งหมด
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

In [ ]:
# 🔹 6. Training Arguments
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
training_args = TrainingArguments(
    output_dir="./results",          # ที่เก็บผลลัพธ์
    evaluation_strategy="epoch",     # ประเมินผลในแต่ละ epoch
    learning_rate=2e-5,              # อัตราการเรียนรู้
    per_device_train_batch_size=8,   # ขนาด batch ในการฝึก
    per_device_eval_batch_size=8,    # ขนาด batch ในการทดสอบ
    num_train_epochs=3,              # จำนวน epochs
    weight_decay=0.01,               # การปรับน้ำหนัก
    logging_dir="./logs",            # ที่เก็บ logs
    logging_steps=10,                # จำนวน step ที่จะบันทึก log
    report_to="none"                 # ปิดการรายงานไปที่ W&B
)

In [ ]:
# 🔹 7. สร้าง Trainer
trainer = Trainer(
    model=model,                     # โมเดลที่ใช้ฝึก
    args=training_args,              # argument ที่กำหนด
    train_dataset=train_dataset,     # ข้อมูลฝึก
    eval_dataset=val_dataset,        # ข้อมูลทดสอบ
    tokenizer=tokenizer,             # tokenizer
    data_collator=data_collator      # data collator
)

In [ ]:
# 🔹 8. ฝึกสอนโมเดล
trainer.train()